In [ ]:
! pip install xlrd

In [1]:
import pandas_profiling as pp
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
! ls -l ../data/original

In [ ]:
! head ../data/original/DB_V2.TXT

## loading the data

In [2]:
# RIZIV dataset 1.8GB
df = pd.read_csv("../data/original/DB_V2.TXT", sep='\t', encoding='latin-1')

/home/ubuntu/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pharma_ref = pd.read_excel("../data/original/PHARMA_REF.xlsx")

In [4]:
reimburse_cat = pd.read_excel("../data/original/reimb category.xlsx")

## df datacleaning

In [5]:
df.drop('Year', axis=1, inplace=True)
df.drop('account_yy_ss', axis=1, inplace=True)

## pharma_ref datacleaning

In [6]:
pharma_ref["drug_code"] = pharma_ref["drug_code"].astype("category")

## reimburse_cat datacleaning

In [7]:
reimburse_cat_dict = {'nd':'other', 'A':'A', 'B':'B', 'Cat 1':'other', 'Cat 2 (A)':'other', 'Cat 3':'other', 'Cat 4':'other',
       'Cat 5 (D)':'other', 'C':'C', 'Cs':'C', 'Cx':'C', 'Cxg':'C', 'D':'D', 'Csg':'C', 'Ag':'A', 'Bg':'B', 'Cg':'C',
       'Forf Ant':'other', 'Nutri Par':'other', 'Br':'other', 'Ar':'other', 'Cr':'C', 'Csr':'C', 'Cxr':'C',
       'Forf Adm':'other', 'Forf BH':'forf', 'V08':'other', 'Fa':'other', 'Fb':'other', 'Forf 1-3':'other',
       'Forf 4-':'other', 'Ri-D11':'other', 'Ri-T1':'other', 'Ri-T2':'other', 'Ri-T3':'other', 'Ri-D5':'other', 'Ri-D7':'other',
       'Ri-D2':'other', 'Ri-D9':'other', 'Ri-D6':'other', 'Ri-D10':'other', 'Ri-D3':'other', 'Ri-D1':'other', 'Ri-D8':'other',
       'Ri-T4':'other', 'Ri-D4':'other', 'Forf PET':'other', '90-A':'A', '90-B':'B', '90-Fa':'other', '90-Fb':'other',
       'Ri-T5':'other', 'Ri-T6':'other', 'Ri-T7':'other', 'Ri-T8':'other', '90-C':'C', '90-Cs':'C', '90-Cx':'C'}

In [8]:
reimburse_cat["reimbt_crit_long"] = reimburse_cat["reimbt_crit_long"].map(reimburse_cat_dict)

In [9]:
# only keep hospitalised data, ignore ambualnt data
reimburse_cat_hosp = reimburse_cat[ (reimburse_cat["patient_cat"] == 'HOSP') ]

In [10]:
# max_pct is highly correlated with min_pct (ρ = 0.98479)
reimburse_cat_hosp.drop('min_pct', axis=1, inplace=True)

/home/ubuntu/venv/lib/python3.5/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
# patient_cat has constant value HOSP
reimburse_cat_hosp.drop('patient_cat', axis=1, inplace=True)

## MERGE inner joins

In [12]:
df = df.merge(pharma_ref, how="left", left_on="drug_code", right_on="drug_code").reset_index()

In [13]:
df = df.merge(reimburse_cat_hosp, how="left", left_on="reimbt_cat_id", right_on="reimbt_cat_id").reset_index()

## df datacleaning phase II

In [14]:
df.drop("level_0", axis=1, inplace=True)
df.drop("index", axis=1, inplace=True)

In [15]:
# set this columns as datetime dtype
df['date_debut_rembourse'] = pd.to_datetime(df['date_debut_rembourse'], format='%Y-%m-%d')

In [ ]:
df = df.astype({'Province':'str',
              'Type':'category',
              'hosp_serv_id':'str',
              'reimbt_cat_id':'int64',
              'drug_code':'int64',
              'realization_date':'str',
              'quantity':'int64',
              'amount_reimb':'float64',
              'amount_not_reimb':'float64',
              'trim_pharma':'int64',
              'produit_pharma':'int64',
              'type_drug_code':'category',
              'famille_produit_pharma':'str',
              'drug_name_aggregated':'str',
              'nom_produit_pharma':'str',
              'conditionnement':'str',
              'mode_administration':'str',
              'nom_firme_pharma':'str',       
              'statut_produit_pharma':'category',
              'code_atc':'str',
              'code_atc_5':'str',
              'code_atc_4':'str',
              'code_atc_3':'str',
              'code_atc_1':'str',
              'DDD':'float64',
              'nombre_prises':'int64',
              'orphan_flag':'bool',
              'chapter_IV_bis_flag':'bool',
              'link_same_tablet':'int64',
              'reimbt_cat_desc_fr':'str',
              'reimbt_cat_desc_nl':'str',
              'reimbt_crit_short':'str',
              'reimbt_cat_acute_yn':'bool',
              'reimbt_cat_chron_yn':'bool',
              'reimbt_cat_psy_yn':'bool',
              'max_pct':'float64',
              'reimbt_cat_fixed_rate_yn':'bool',
              'relative_care_yn':'bool',
              'ami_ziv_amount_null_yn':'bool',
              'not_reimbursed_null_yn':'bool',
              'fee_cat':'category',
              'reimbt_cat_id_pk':'int64'})

In [16]:
df["realization_date"] = df["realization_date"].astype('str')

In [17]:
df = df[ (df["realization_date"].str[:4] == '2016')]

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1542611 entries, 22833075 to 26009296
Data columns (total 46 columns):
Province                    1542611 non-null object
Type                        1542611 non-null object
hosp_serv_id                1542611 non-null object
reimbt_cat_id               1542611 non-null int64
drug_code                   1542611 non-null object
realization_date            1542611 non-null object
quantity                    1542611 non-null int64
amount_reimb                1542611 non-null float64
amount_not_reimb            1542611 non-null float64
trim_pharma                 1542611 non-null float64
produit_pharma              1542611 non-null float64
type_drug_code              1538323 non-null object
famille_produit_pharma      1538323 non-null object
drug_name_aggregated        1538323 non-null object
nom_produit_pharma          1538323 non-null object
conditionnement             1509986 non-null object
mode_administration         1510761 non-null 

## output result

In [20]:
pp.ProfileReport(df).to_file(outputfile="../data/report/DB_V4_2016_report.html")

/home/ubuntu/venv/lib/python3.5/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/ubuntu/venv/lib/python3.5/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/ubuntu/venv/lib/python3.5/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/ubuntu/venv/lib/python3.5/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/home/ubuntu/venv/lib/python3.5/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['

In [ ]:
#df.to_csv("../data/dataset/DB_V4.csv", index=False)

In [21]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1542611 entries, 22833075 to 26009296
Data columns (total 46 columns):
Province                    1542611 non-null object
Type                        1542611 non-null object
hosp_serv_id                1542611 non-null object
reimbt_cat_id               1542611 non-null int64
drug_code                   1542611 non-null object
realization_date            1542611 non-null object
quantity                    1542611 non-null int64
amount_reimb                1542611 non-null float64
amount_not_reimb            1542611 non-null float64
trim_pharma                 1542611 non-null float64
produit_pharma              1542611 non-null float64
type_drug_code              1538323 non-null object
famille_produit_pharma      1538323 non-null object
drug_name_aggregated        1538323 non-null object
nom_produit_pharma          1538323 non-null object
conditionnement             1509986 non-null object
mode_administration         1510761 non-null 

In [ ]:
#import pandas_profiling as pp

In [ ]:
#pp.ProfileReport(pharma_ref)

In [ ]:
#df["realization_date"] = df["realization_date"].astype('str')

In [ ]:
#(df["realization_date"].str[:4] == '2016').value_counts()

In [22]:
df.columns

Index(['Province', 'Type', 'hosp_serv_id', 'reimbt_cat_id', 'drug_code',
       'realization_date', 'quantity', 'amount_reimb', 'amount_not_reimb',
       'trim_pharma', 'produit_pharma', 'type_drug_code',
       'famille_produit_pharma', 'drug_name_aggregated', 'nom_produit_pharma',
       'conditionnement', 'mode_administration', 'nom_firme_pharma',
       'date_debut_rembourse', 'statut_produit_pharma', 'code_atc',
       'code_atc_5', 'code_atc_4', 'code_atc_3', 'code_atc_1', 'DDD',
       'nombre_prises', 'orphan_flag', 'chapter_IV_bis_flag',
       'link_same_tablet', 'dbegin', 'dend', 'reimbt_cat_desc_fr',
       'reimbt_cat_desc_nl', 'reimbt_crit_long', 'reimbt_crit_short',
       'reimbt_cat_acute_yn', 'reimbt_cat_chron_yn', 'reimbt_cat_psy_yn',
       'max_pct', 'reimbt_cat_fixed_rate_yn', 'relative_care_yn',
       'ami_ziv_amount_null_yn', 'not_reimbursed_null_yn', 'fee_cat',
       'reimbt_cat_id_pk'],
      dtype='object')

In [24]:
# keep these columns:
df = df[['Province', 'Type', 'hosp_serv_id', 'reimbt_cat_id', 'drug_code',
       'realization_date', 'quantity', 'amount_reimb', 'amount_not_reimb',
       'trim_pharma', 'produit_pharma', 'type_drug_code',
       'famille_produit_pharma', 'drug_name_aggregated',
       'conditionnement', 'mode_administration',
       'date_debut_rembourse', 'statut_produit_pharma', 'code_atc',
       'code_atc_5', 'code_atc_4', 'code_atc_3', 'code_atc_1', 'DDD',
       'nombre_prises', 'orphan_flag', 'chapter_IV_bis_flag',
       'link_same_tablet', 'dbegin', 'dend', 'reimbt_cat_desc_fr',
       'reimbt_cat_desc_nl', 'reimbt_crit_long', 'reimbt_crit_short',
       'reimbt_cat_fixed_rate_yn', 'fee_cat']]

In [29]:
df["reimbt_cat_fixed_rate_yn"] = df["reimbt_cat_fixed_rate_yn"].astype('bool')

In [37]:
df_work = df[ (df["reimbt_cat_fixed_rate_yn"] == True) ]

In [38]:
df_work.to_csv("../data/dataset/not_reimb_workfile_2016.csv", index=False)

In [40]:
df_work["Type"].value_counts()

Général          953388
Psychiatrique      6046
Name: Type, dtype: int64

In [63]:
df_work["amount_reimb_div_quantity"] = df_work["amount_reimb"] / df_work["quantity"]

/home/ubuntu/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#df.groupby('col1')['col2'].apply(lambda x: x)

In [41]:
df_work = df_work[ (df_work["Type"] == "Général") ]

In [68]:
df_work.groupby(["code_atc"])["amount_reimb_div_quantity"].sum().value_counts(ascending=False)

 0.000000e+00    520
 inf             101
-inf              13
 5.824304e+00      1
 5.003638e+02      1
 4.261144e+00      1
 4.202902e+01      1
 4.718318e+01      1
 3.732954e+02      1
 6.304515e+01      1
 5.873456e+02      1
 8.036152e+00      1
 9.219011e+04      1
 2.022065e+04      1
 9.989872e+03      1
 3.684756e+01      1
 1.139832e+01      1
 4.038665e+03      1
 2.662045e-01      1
 2.348433e+01      1
 9.977944e+03      1
 7.716923e+01      1
 2.796620e+03      1
 7.934648e+00      1
 8.849044e+01      1
 1.206362e+02      1
 2.878000e+01      1
 1.138605e+04      1
 3.721350e+03      1
 2.942728e+02      1
                ... 
 3.865374e+04      1
 1.041316e+06      1
 2.406465e+01      1
 7.219843e+00      1
 4.166468e+01      1
 3.284791e+01      1
 2.573558e+00      1
 4.382187e+00      1
 1.281722e+05      1
 4.841690e+02      1
 4.057019e+01      1
 6.042004e+03      1
 6.111199e+01      1
 2.092393e+05      1
 7.775758e+02      1
 1.050866e+04      1
 2.576000e+02

In [46]:
df_work.groupby(["code_atc"]).value_counts()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'